# E-commerce Analytics Warehouse — Exploratory Analysis

This notebook connects to the `ecommerce_warehouse` PostgreSQL database and walks through a series of analytics commonly
used in an e-commerce setting:

- Overall business KPIs
- Revenue trends and seasonality
- Channel performance
- Customer lifetime value (LTV) and RFM analysis
- Cohort retention
- Category profitability
- Market basket / product association analysis
- Sales forecasting
- Customer segmentation with K-Means

The goal is to demonstrate both the underlying data warehouse design and the kind of insights it enables.


## 1. Setup and Database Connection


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2
import os

# Plot settings (optional)
plt.rcParams["figure.figsize"] = (10, 5)

DB_CONFIG = {
    "host": "localhost",
    "port": 5432,
    "dbname": "ecommerce_warehouse",
    "user": "postgres",
    "password": os.getenv("PG_PASSWORD", ""),
}

def get_connection():
    return psycopg2.connect(**DB_CONFIG)

def run_query(sql, params=None):
    with get_connection() as conn:
        return pd.read_sql(sql, conn, params=params)

## 2. Core Business KPIs

First, we compute a few high-level metrics based on completed orders:

- Total number of orders
- Number of active (purchasing) customers
- Total revenue
- Average order value (AOV)


In [ ]:
kpi_query = """
SELECT
    COUNT(DISTINCT o.order_id) AS total_orders,
    COUNT(DISTINCT o.customer_id) AS active_customers,
    SUM(o.order_total) AS total_revenue,
    AVG(o.order_total) AS avg_order_value
FROM fact_order o
WHERE o.order_status = 'Completed';
"""

kpis = run_query(kpi_query)
kpis


## 3. Revenue Trends Over Time

We begin with a view of total revenue aggregated by year and month. This helps us understand growth patterns and
seasonality at a high level.


In [ ]:
rev_time_query = """
SELECT
    year,
    month,
    SUM(total_revenue) AS total_revenue
FROM vw_daily_revenue
GROUP BY year, month
ORDER BY year, month;
"""

rev_time = run_query(rev_time_query)
rev_time["year_month"] = rev_time["year"].astype(str) + "-" + rev_time["month"].astype(str).str.zfill(2)

plt.plot(rev_time["year_month"], rev_time["total_revenue"])
plt.xticks(rotation=45, ha="right")
plt.title("Monthly Revenue Over Time")
plt.xlabel("Year-Month")
plt.ylabel("Revenue")
plt.tight_layout()
plt.show()


## 4. Revenue by Marketing Channel

Next, we examine how different acquisition channels contribute to revenue. This is based on the `vw_revenue_by_channel`
view, which aggregates completed orders by channel and month.


In [ ]:
rev_channel_query = """
SELECT
    channel_name,
    SUM(total_revenue) AS revenue
FROM vw_revenue_by_channel
GROUP BY channel_name
ORDER BY revenue DESC;
"""

rev_channel = run_query(rev_channel_query)
rev_channel


In [ ]:
plt.bar(rev_channel["channel_name"], rev_channel["revenue"])
plt.xticks(rotation=45, ha="right")
plt.title("Total Revenue by Channel")
plt.ylabel("Revenue")
plt.tight_layout()
plt.show()


## 5. Customer Lifetime Value (LTV) Segments

Using the `vw_customer_ltv_segments` view, we bucket customers into four LTV tiers (Bronze, Silver, Gold, Platinum)
based on their total revenue. This section shows how many customers fall into each tier and how much revenue each tier
contributes.


In [ ]:
ltv_seg_query = """
SELECT
    ltv_segment,
    COUNT(*) AS customers,
    ROUND(AVG(total_revenue), 2) AS avg_revenue,
    ROUND(SUM(total_revenue), 2) AS total_revenue
FROM vw_customer_ltv_segments
GROUP BY ltv_segment
ORDER BY total_revenue DESC;
"""

ltv_seg = run_query(ltv_seg_query)
ltv_seg


In [ ]:
plt.bar(ltv_seg["ltv_segment"], ltv_seg["total_revenue"])
plt.title("Total Revenue by LTV Segment")
plt.xlabel("LTV Segment")
plt.ylabel("Total Revenue")
plt.tight_layout()
plt.show()


## 6. RFM Analysis (Recency, Frequency, Monetary Value)

Here we calculate RFM scores per customer, which summarize:

- **Recency**: how many days since the last order
- **Frequency**: how many orders the customer has placed
- **Monetary value**: total revenue from that customer

We use these to identify high-value and at-risk customers.


In [ ]:
rfm_top_query = """
SELECT
    customer_key,
    first_name,
    last_name,
    segment,
    recency_days,
    frequency,
    monetary_value,
    recency_score,
    frequency_score,
    monetary_score,
    rfm_score,
    rfm_code
FROM vw_customer_rfm_scores
ORDER BY rfm_score DESC, monetary_value DESC
LIMIT 20;
"""

rfm_top = run_query(rfm_top_query)
rfm_top


In [ ]:
rfm_all_query = """
SELECT
    frequency,
    monetary_value,
    recency_days
FROM vw_customer_rfm_scores;
"""

rfm_all = run_query(rfm_all_query)

plt.scatter(
    rfm_all["frequency"],
    rfm_all["monetary_value"],
    s=30,
    alpha=0.5,
)
plt.xlabel("Frequency (Number of Orders)")
plt.ylabel("Monetary Value (Total Spend)")
plt.title("Customer Frequency vs Monetary Value")
plt.tight_layout()
plt.show()


## 7. Cohort Retention Analysis

Cohorts are defined by the month of a customer's first order. For each cohort, we track what percentage of customers are
active in each subsequent month. The heatmap below shows retention curves over time.


In [ ]:
cohort_query = """
SELECT
    cohort_month,
    months_since_cohort_start,
    retention_percent
FROM vw_cohort_retention
ORDER BY cohort_month, months_since_cohort_start;
"""

cohort = run_query(cohort_query)
cohort.head()


In [ ]:
cohort_pivot = cohort.pivot_table(
    index="cohort_month",
    columns="months_since_cohort_start",
    values="retention_percent"
).fillna(0)

cohort_pivot


In [ ]:
plt.figure(figsize=(10, 6))
plt.imshow(cohort_pivot.values, aspect="auto")
plt.colorbar(label="Retention (%)")

plt.xticks(
    range(cohort_pivot.shape[1]),
    cohort_pivot.columns,
)
plt.yticks(
    range(cohort_pivot.shape[0]),
    [d.strftime("%Y-%m") for d in cohort_pivot.index]
)

plt.xlabel("Months Since Cohort Start")
plt.ylabel("Cohort Month")
plt.title("Customer Cohort Retention (%)")
plt.tight_layout()
plt.show()


## 8. Category Profitability (Revenue, Cost, Margin)

Using the `vw_category_margin_monthly` view, we analyze revenue, cost, and gross margin percentage over time for each
product category. This highlights which categories are driving profit versus just top-line revenue.


In [ ]:
margin_query = """
SELECT
    year,
    month,
    category,
    subcategory,
    revenue,
    total_cost,
    gross_profit,
    gross_margin_percent
FROM vw_category_margin_monthly
ORDER BY year, month, category, subcategory;
"""

margin = run_query(margin_query)
margin.head()


In [ ]:
# Pick top 3 categories by total revenue
top_cats = (
    margin.groupby("category")["revenue"]
    .sum()
    .sort_values(ascending=False)
    .head(3)
    .index
)

margin_top = margin[margin["category"].isin(top_cats)].copy()
margin_top["year_month"] = (
    margin_top["year"].astype(str) + "-" + margin_top["month"].astype(str).str.zfill(2)
)

plt.figure(figsize=(10, 5))
for cat in top_cats:
    cat_data = (
        margin_top[margin_top["category"] == cat]
        .sort_values(["year", "month"])
    )
    plt.plot(cat_data["year_month"], cat_data["gross_margin_percent"], label=cat)

plt.xticks(rotation=45, ha="right")
plt.xlabel("Year-Month")
plt.ylabel("Gross Margin (%)")
plt.title("Gross Margin % Over Time by Top Categories")
plt.legend()
plt.tight_layout()
plt.show()


## 9. Product Associations (Market Basket Analysis)

We compute product pairs that frequently appear in the same order and derive classic association rule metrics:

- **Support**: how often the pair occurs relative to all orders  
- **Confidence**: probability of buying B given A  
- **Lift**: how much more likely B is purchased with A than by chance

This is the basis for “Frequently bought together” style recommendations.


In [ ]:
pairs_query = """
SELECT
    product_name_1,
    product_name_2,
    cooccurrence_orders
FROM vw_product_pairs
ORDER BY cooccurrence_orders DESC
LIMIT 20;
"""

product_pairs = run_query(pairs_query)
product_pairs


In [ ]:
pair_labels = (
    product_pairs["product_name_1"] + " & " + product_pairs["product_name_2"]
)

plt.figure(figsize=(10, 6))
plt.barh(pair_labels, product_pairs["cooccurrence_orders"])
plt.xlabel("Number of Orders Together")
plt.title("Top Product Pairs (Frequently Bought Together)")
plt.gca().invert_yaxis()  # largest at top
plt.tight_layout()
plt.show()


In [ ]:
assoc_query = """
SELECT
    product_name_1,
    product_name_2,
    cooccurrence_orders,
    ROUND(support, 4) AS support,
    ROUND(confidence_1_to_2, 4) AS confidence_1_to_2,
    ROUND(lift, 4) AS lift
FROM vw_product_association_rules
ORDER BY lift DESC
LIMIT 20;
"""

assoc = run_query(assoc_query)
assoc


In [ ]:
pair_labels = assoc["product_name_1"] + " → " + assoc["product_name_2"]

plt.figure(figsize=(10, 6))
plt.barh(pair_labels, assoc["lift"])
plt.xlabel("Lift")
plt.title("Top Product Association Rules by Lift")
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()


## 10. Revenue Forecasting (Moving Average Baseline)

As a simple baseline forecast, we compute monthly revenue, apply a 3-month moving average for smoothing, and then extend
that out as a naive forecast for the next few months. This isn't a full forecasting model, but it demonstrates how the
warehouse can feed time-series forecasting workflows.


In [ ]:
rev_forecast_query = """
SELECT
    d.date_actual::date AS date,
    SUM(o.order_total) AS total_revenue
FROM fact_order o
JOIN dim_date d ON d.date_id = o.order_date_id
WHERE o.order_status = 'Completed'
GROUP BY d.date_actual
ORDER BY d.date_actual;
"""

rev_daily = run_query(rev_forecast_query)

# ✅ Ensure date is datetime
rev_daily["date"] = pd.to_datetime(rev_daily["date"])

# Aggregate to month level
rev_monthly = (
    rev_daily
    .groupby(rev_daily["date"].dt.to_period("M"))
    .agg(total_revenue=("total_revenue", "sum"))
    .reset_index()
)

rev_monthly["date"] = rev_monthly["date"].dt.to_timestamp()
rev_monthly = rev_monthly.sort_values("date")

# 3-month moving average
rev_monthly["ma_3"] = rev_monthly["total_revenue"].rolling(window=3).mean()
rev_monthly.tail()


In [ ]:
plt.plot(rev_monthly["date"], rev_monthly["total_revenue"], label="Actual")
plt.plot(rev_monthly["date"], rev_monthly["ma_3"], label="3-Month Moving Average")
plt.title("Monthly Revenue with 3-Month Moving Average")
plt.xlabel("Month")
plt.ylabel("Revenue")
plt.legend()
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()


In [ ]:
last_date = rev_monthly["date"].max()
last_ma = rev_monthly["ma_3"].iloc[-1]

future_months = pd.date_range(last_date + pd.offsets.MonthBegin(1), periods=6, freq="MS")
future_forecast = pd.DataFrame({
    "date": future_months,
    "forecast_revenue": [last_ma] * len(future_months)
})

plt.plot(rev_monthly["date"], rev_monthly["total_revenue"], label="Actual")
plt.plot(rev_monthly["date"], rev_monthly["ma_3"], label="3-Month MA")
plt.plot(future_forecast["date"], future_forecast["forecast_revenue"], "--", label="Naive Forecast")
plt.title("Revenue with Simple Moving Average Forecast")
plt.xlabel("Month")
plt.ylabel("Revenue")
plt.legend()
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()


## 11. Category Seasonality

To understand seasonality, we aggregate revenue by category and calendar month and visualize the result as a heatmap.
Brighter cells indicate higher revenue for that category-month combination.


In [ ]:
seasonality_query = """
SELECT
    month,
    category,
    SUM(revenue) AS total_revenue
FROM vw_category_margin_monthly
GROUP BY month, category
ORDER BY month, category;
"""

seasonality = run_query(seasonality_query)
seasonality.head()


In [ ]:
season_pivot = seasonality.pivot_table(
    index="category",
    columns="month",
    values="total_revenue"
).fillna(0)

season_pivot


In [ ]:
plt.figure(figsize=(10, 6))
plt.imshow(season_pivot.values, aspect="auto")
plt.colorbar(label="Revenue")

plt.xticks(
    range(season_pivot.shape[1]),
    season_pivot.columns
)
plt.yticks(
    range(season_pivot.shape[0]),
    season_pivot.index
)

plt.xlabel("Month")
plt.ylabel("Category")
plt.title("Category Seasonality (Revenue by Month)")
plt.tight_layout()
plt.show()


## 12. Customer Segmentation with K-Means (RFM Features)

Finally, we use unsupervised learning (K-Means clustering) on RFM features:

- Recency (days since last order)
- Frequency (number of orders)
- Monetary value (total spend)

to group customers into behavioral segments. This can inform differential marketing strategies.


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

rfm_features_query = """
SELECT
    customer_id,
    recency_days,
    frequency,
    monetary_value
FROM vw_customer_rfm_base;
"""

rfm_features = run_query(rfm_features_query)
rfm_features.head()


In [ ]:
X = rfm_features[["recency_days", "frequency", "monetary_value"]].values

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

kmeans = KMeans(n_clusters=4, random_state=42, n_init=10)
rfm_features["cluster"] = kmeans.fit_predict(X_scaled)

rfm_features["cluster"].value_counts()


In [ ]:
cluster_summary = (
    rfm_features
    .groupby("cluster")
    .agg(
        avg_recency=("recency_days", "mean"),
        avg_frequency=("frequency", "mean"),
        avg_monetary=("monetary_value", "mean"),
        customers=("customer_id", "count")
    )
    .sort_values("avg_monetary", ascending=False)
)
cluster_summary


In [ ]:
plt.figure(figsize=(8, 6))
for cluster_id in sorted(rfm_features["cluster"].unique()):
    cluster_data = rfm_features[rfm_features["cluster"] == cluster_id]
    plt.scatter(
        cluster_data["frequency"],
        cluster_data["monetary_value"],
        s=30,
        alpha=0.6,
        label=f"Cluster {cluster_id}"
    )

plt.xlabel("Frequency (number of orders)")
plt.ylabel("Monetary Value (total spend)")
plt.title("Customer Segmentation via K-Means (RFM)")
plt.legend()
plt.tight_layout()
plt.show()
